<a href="https://colab.research.google.com/github/JasonCFowler/DigitalRepos/blob/master/MARC_CSV_Merge_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [96]:
# Section 1: Install Pymarc
# -------------------------
!pip install pymarc
# Section 2: Import Required Libraries
# ------------------------------------
import csv
import pymarc

In [98]:
# Section 3: Define Function to Add 949 Field
# -------------------------------------------
def add_949_to_marc(marc_record, data_from_csv):
    # Subfield h is the OCLC branch, subfield l is the shelving location,
    # and subfield p is the fund code
    field = pymarc.Field(
        tag='949',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='h', value='5053'),
            pymarc.Subfield(code='l', value='Unspecified'),
            pymarc.Subfield(code='p', value='EBOOKS OTHER')
        ]
    )
    marc_record.add_field(field)

In [99]:
# Section 4: Define Function to Add 980 Field
# -------------------------------------------
def add_980_to_marc(marc_record, data_from_csv):
    # Subfield b is the unit price, subfield c is the service charge,
    # subfield h is the tax percentage, subfield i is the discount percentage,
    # subfield k is the isbn, and subfield z is the currency of the purchase order
    field = pymarc.Field(
        tag='980',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='b', value=data_from_csv.get('UnitPrice', ' ')),
            pymarc.Subfield(code='i', value=data_from_csv.get('DiscountPercentage', ' ')),
            pymarc.Subfield(code='k', value=data_from_csv.get('ISBN', ' ')),
            pymarc.Subfield(code='z', value='USD')
        ]
    )
    marc_record.add_field(field)

In [100]:
# Section 5: Define Function to Add 981 Field
# -------------------------------------------
def add_981_to_marc(marc_record, data_from_csv):
    # Subfield a is the KB Collection Identifier,
    # subfield b is the KB Title Identifier, and
    # subfield n is the Vendor Order Number
    field = pymarc.Field(
        tag='981',
        indicators=[' ', ' '],
        subfields=[
            pymarc.Subfield(code='a', value='netlibrary.ebooks'),
            pymarc.Subfield(code='b', value=data_from_csv.get('KBTitleIdentifier', ' ')),
            pymarc.Subfield(code='n', value=data_from_csv.get('VendorOrderNumber', ' '))
        ]
    )
    marc_record.add_field(field)


In [112]:
# Section 6: Function to match and update MARC records based on ISBN
def update_marc_records(marc_file_path, csv_file_path):
    # Read MARC records
    with open(marc_file_path, 'rb') as marc_file:
        reader = MARCReader(marc_file)
        records = list(reader)

    # Read CSV data
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            isbn = row['\ufeffISBN']  # Modify this line

            # Match MARC record using ISBN
            matched_record = None
            for record in records:
              # Add your matching criteria here
              isbns = [field['a'] for field in record.get_fields('020') if 'a' in field]
              print(f"MARC Record ISBNs: {isbns}")
              print(f"ISBN to Match: {isbn}")
              if any(isbn == record_isbn for record_isbn in isbns):
                matched_record = record
                break

            # Update MARC record with 949, 980, and 981 fields
            if matched_record:
                print(f"Processing ISBN: {isbn}")
                try:
                    add_949_to_marc(matched_record, row)
                    add_980_to_marc(matched_record, row)
                    add_981_to_marc(matched_record, row)
                except KeyError as e:
                    print(f"KeyError: {e}")

    # Write updated MARC records to a new file
    with open('updated_marc_records.mrc', 'wb') as updated_marc_file:
        for record in records:
            updated_marc_file.write(record.as_marc())



In [113]:
# Section 7: Execute
if __name__ == "__main__":
    # Specify file paths
    marc_file_path = 'OrderID_4673997_20231114_UTF8.mrc'
    csv_file_path = 'your_csv_file-test.csv'

    # Call the function to update MARC records
    update_marc_records(marc_file_path, csv_file_path)


MARC Record ISBNs: ['1087764939', '9781087764931']
ISBN to Match: 9781087764931
Processing ISBN: 9781087764931
MARC Record ISBNs: ['1087764939', '9781087764931']
ISBN to Match: 9781629958545
MARC Record ISBNs: ['9781629958545', '1629958549']
ISBN to Match: 9781629958545
Processing ISBN: 9781629958545
MARC Record ISBNs: ['1087764939', '9781087764931']
ISBN to Match: 9781629955346
MARC Record ISBNs: ['9781629958545', '1629958549']
ISBN to Match: 9781629955346
MARC Record ISBNs: ['9781629955346', '1629955345']
ISBN to Match: 9781629955346
Processing ISBN: 9781629955346


In [104]:
import csv

csv_file_path = 'your_csv_file-test.csv'  # Replace with your actual CSV file path

# Print the first few rows of the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    for i, row in enumerate(csv_reader):
        print(row)
        if i == 5:  # Print only the first 5 rows for inspection
            break

['\ufeffISBN', 'KB_Collection_ID', 'ShelvingLocation', 'FundCode', 'UnitPrice', 'ServiceCharge', 'TaxPercentage', 'DiscountPercentage', 'KB_Title_ID', 'Vendor_Order_Number', 'Title', 'Author', 'Date', 'Publisher']
['9781087764931', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$29.99 ', '0', '7.5', '30', 'KB_Title_456', 'VN123456', 'Pastoring Small Towns', '', '2022', 'Sample Publisher']
['9781629958545', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$14.99 ', '0', '7.5', '30', 'KB_Title_012', 'VN789012', 'Alvin Plantinga', '', '2021', 'Another Publisher']
['9781629955346', 'netlibrary.ebooks', 'Unspecified', 'EBOOKS OTHER', '$28.49 ', '0', '7.5', '30', 'KB_Title_789', 'VN456789', "The Shepherd's Toolbox: Advancing Your Church's Shepherding Ministry", '', '2020', 'Yet Another Publisher']
